# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [1]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [3]:
FAHRENHEIT_OFFSET = 32
FAHRENHEIT_TO_CELSIUS = 5 / 9

def fahrenheit_to_celsius(fahrenheit):
    return (fahrenheit - FAHRENHEIT_OFFSET) * FAHRENHEIT_TO_CELSIUS

def read_temperatures(filename):
    temperatures = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            fahrenheit = float(row[0])
            celsius = fahrenheit_to_celsius(fahrenheit)
            temperatures.append(celsius)
    return temperatures

def calculate_stats(temperatures):
    average = sum(temperatures) / len(temperatures)
    return average, min(temperatures), max(temperatures)

def write_to_csv(filename, temperatures):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for t in temperatures:
            writer.writerow([t])

def main(input_file, output_file):
    temps = read_temperatures(input_file)
    average, minimum, maximum = calculate_stats(temps)

    print("Statistics:")
    print(f"Average: {average:.2f}°C")
    print(f"Minimum: {minimum:.2f}°C")
    print(f"Maximum: {maximum:.2f}°C")

    write_to_csv(output_file, temps)


### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [6]:
from unittest.mock import mock_open, patch
from io import StringIO

class UnitTestCase(unittest.TestCase):
    def test_fahrenheit_to_celsius(self):
        self.assertAlmostEqual(fahrenheit_to_celsius(32), 0.0)
        self.assertAlmostEqual(fahrenheit_to_celsius(212), 100.0)
        self.assertAlmostEqual(fahrenheit_to_celsius(77), 25.0, places=6)

    def test_calculate_stats(self):
        avg, mn, mx = calculate_stats([0.0, 10.0, 20.0])
        self.assertAlmostEqual(avg, 10.0)
        self.assertEqual(mn, 0.0)
        self.assertEqual(mx, 20.0)


class IntegrationTestCase(unittest.TestCase):
    def test_main_reads_converts_and_writes_csv(self):
        input_csv = "Temperature (F)\n78.5\n81.2\n75.9\n"
        m_in = mock_open(read_data = input_csv)
        m_out = mock_open()

        with patch("builtins.open", side_effect=[m_in.return_value, m_out.return_value]):
            main("in.csv", "out.csv")

        written = "".join(call.args[0] for call in m_out().write.mock_calls)
        rows = list(csv.reader(StringIO(written)))
        self.assertEqual(rows[0], ["Temperature (C)"])

        values = [float(r[0]) for r in rows[1:]]
        self.assertAlmostEqual(values[0], fahrenheit_to_celsius(78.5))
        self.assertAlmostEqual(values[1], fahrenheit_to_celsius(81.2))
        self.assertAlmostEqual(values[2], fahrenheit_to_celsius(75.9))


unittest.main(argv=[''], exit = False)


...
----------------------------------------------------------------------
Ran 3 tests in 0.017s

OK


Statistics:
Average: 25.85°C
Minimum: 24.39°C
Maximum: 27.33°C
